In [2]:
from info import * 
from preprocess import *
import utils

/Users/matthew/opt/anaconda3/envs/social_navigation_task/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/matthew/opt/anaconda3/envs/social_navigation_task/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/matthew/opt/anaconda3/envs/social_navigation_task/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [2]:
xlsxs = ['../data/example_subject/snt_583_narrative_data.xlsx']
behavior = ComputeBehavior(file_path=xlsxs[0]).run()
behavior

/Users/matthew/opt/anaconda3/envs/social_navigation_task/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,scene_num,decision_num,char_decision_num,dimension,char_role_num,onset,button_press,decision,affil,power,...,pov_distance,neu3d_angle,pov3d_angle,perimeter,area,volume,Q1_overlap,Q2_overlap,Q3_overlap,Q4_overlap
0,1,1,1,affil,1,116.830780,2,-1,-1.0,0.0,...,7.000000,2.070143,1.570796,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,2,affil,1,131.725189,3,-1,-1.0,0.0,...,8.000000,2.111216,1.570796,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,3,affil,1,151.223053,2,-1,-1.0,0.0,...,9.000000,2.158799,1.570796,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,4,4,power,1,176.024872,2,-1,0.0,-1.0,...,9.055385,2.176341,1.637104,5.236068,1.0,NaN,0.000000,0.000000,1.000000,0.000000
4,2,5,1,affil,2,198.424835,2,-1,-1.0,0.0,...,7.000000,2.070143,1.570796,5.236068,1.0,NaN,0.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,17,59,12,power,4,1571.144531,3,-1,0.0,-1.0,...,6.324555,1.570796,1.249046,24.808145,40.5,254.000000,0.129630,0.222222,0.469136,0.179012
59,18,60,9,power,3,1591.643066,2,1,0.0,1.0,...,3.000000,0.463648,1.570796,24.941561,41.5,260.333333,0.144578,0.216867,0.457831,0.180723
60,18,61,10,power,3,1608.522217,2,1,0.0,1.0,...,3.162278,0.451027,1.266104,25.164437,42.5,266.666667,0.164706,0.211765,0.447059,0.176471
61,19,62,11,power,3,1629.921509,2,1,0.0,1.0,...,3.605551,0.608110,0.991837,25.572077,43.5,273.333333,0.183908,0.206897,0.436782,0.172414


## Tests to incorporate

In [6]:
import random
compute = ComputeBehavior(file_path=None)

rand_decs = np.array([random.choice([-1,1]) for _ in range(10)]).reshape(-2,2)
resp_mask = rand_decs != 0
coords = np.cumsum(rand_decs, axis=0)    
cum_mean, cum_count = compute.cumulative_mean(rand_decs, resp_mask)
cum_mean[-1,:]


array([-0.2,  0.6])

In [7]:
coords[-1,:] / cum_count[-1,:]

array([-0.2,  0.6])

In [55]:
# testing the make_3d function:

U = np.random.randint(1,10, size=(3,2)) # all the outputs should have this shape
V = np.random.randint(1,10, size=(2,))
ori = np.array([[0,0]])

## funciton: 
if V.ndim == 2:   V = V[0]
if ori.ndim == 1: ori = ori[np.newaxis]

num = np.arange(1, len(U) + 1)[:,np.newaxis]
U   = np.concatenate([U, num], axis=1) # changes w/ num of interactions
V   = np.repeat(np.hstack([V, len(U)])[np.newaxis], len(U), axis=0) # fixed
ori = np.concatenate([np.repeat(ori, len(U), axis=0), num], axis=1) # changes w/ num of interactions   
# ori & U should have z-axis coordinates that increase sequentially; V should have a fixed z-axis coordinate 

In [2]:
# TODO: concatenate the columns in one go for the df

from pathlib import Path 
import csv
import numpy as np
import pandas as pd
import os
import sys
import scipy.spatial

sys.path.insert(0, '../../../toolbox/toolbox')
from circ_stats import * 
from matrices import *

# should probably import toolbox so circ_stats etc is available...

In [8]:

def by_character(variable):
    ''' return variable name for each character '''
    return [variable + '_' + character_roles[c] for c in range(5)]
    

def chars_df(df, var_name):
    ''' subset a df with the different character values for a variable'''
    return df[[var_name + '_' + cr for cr in character_roles]]


def get_decision_sequence(df):
    ''' grab decisions from a summary df'''
    return df.loc[0, ['decisions_' + "{:02d}".format(t+1) for t in range(63)]]


def repeat_array_cols(X, repeats=2):
    return np.transpose([X] * repeats)


# Log to more detailed xlsx

In [14]:
# file_path = str(Path(f'{pkg_dir}/../data/example_subject/snt_18001.log'))
# experimenter = 'KB'
# parse_log(file_path, experimenter, output_timing=True, out_dir=None)

# Behavior

In [87]:
file_path = Path(f'{pkg_dir}/../data/example_subject/snt_18001.xlsx')
compute_behavior(file_path, out_dir=f'{pkg_dir}/../data/example_subject/')

# for RDVs

In [217]:
from itertools import compress, product, combinations 
def combos(arr, k=2):
    """ 
        arr: np.array to get combos from
        r: num of combos
    """
    return list(combinations(arr, k))

In [242]:
file_path = Path(f'{pkg_dir}/../data/example_subject/snt_18001_behavior.xlsx')
compute_rdvs(file_path, metric='euclidean', out_dir=None)

# summarize all subjs

In [293]:
file_paths = [f'{pkg_dir}/../data/example_subject/snt_18001_behavior.xlsx', f'{pkg_dir}/../data/example_subject/snt_18001_behavior.xlsx']
out_dir=f'{pkg_dir}/preprocessed_behavior/'

summarize_behavior(file_paths, out_dir=out_dir)

/Users/matthew/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


/Users/matthew/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
